# Salescope - Comprehensive Statistical Analysis of Walmart Sales Data

**Developed and Analysed by:** [Srikar MK](https://www.linkedin.com/in/srikarmk/) & [Alekhya Bulusu](https://www.linkedin.com/in/alekhyabulusu/)

This notebook provides a detailed statistical analysis of the Walmart sales dataset, including descriptive statistics, hypothesis testing, correlation analysis, and advanced statistical modeling.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, pearsonr, spearmanr, ttest_ind, mannwhitneyu
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline


In [ ]:
df = pd.read_csv('Walmart_Sales_Data.csv')
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
df.head()


## 1. Data Preprocessing and Feature Engineering


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time
df['hour'] = pd.to_datetime(df['Time'].astype(str), format='%H:%M:%S').dt.hour
df['day_name'] = df['Date'].dt.day_name()
df['month_name'] = df['Date'].dt.month_name()
df['day_of_week'] = df['Date'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6])

def time_of_day(t):
    if t >= pd.to_datetime('05:00:00').time() and t < pd.to_datetime('12:00:00').time():
        return 'Morning'
    elif t >= pd.to_datetime('12:00:00').time() and t < pd.to_datetime('17:00:00').time():
        return 'Afternoon'
    else:
        return 'Evening'

df['time_of_day'] = df['Time'].apply(time_of_day)

numeric_columns = ['Unit price', 'Quantity', 'Tax 5%', 'Total', 'cogs', 'gross margin percentage', 'gross income', 'Rating']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("Data preprocessing completed!")
print(f"Dataset shape after preprocessing: {df.shape}")


## 2. Descriptive Statistics


In [ ]:
print("=== DESCRIPTIVE STATISTICS ===")
print("\n1. Dataset Overview:")
print(f"Total records: {len(df):,}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Number of unique customers: {df['Invoice ID'].nunique():,}")
print(f"Number of branches: {df['Branch'].nunique()}")
print(f"Number of cities: {df['City'].nunique()}")

print("\n2. Revenue Statistics:")
print(f"Total revenue: ${df['Total'].sum():,.2f}")
print(f"Average transaction value: ${df['Total'].mean():.2f}")
print(f"Median transaction value: ${df['Total'].median():.2f}")
print(f"Standard deviation: ${df['Total'].std():.2f}")
print(f"Min transaction: ${df['Total'].min():.2f}")
print(f"Max transaction: ${df['Total'].max():.2f}")


In [ ]:
print("\n3. Detailed Statistics for Numeric Variables:")
desc_stats = df[numeric_columns].describe()
print(desc_stats.round(2))


In [ ]:
print("\n4. Categorical Variables Distribution:")
categorical_vars = ['Branch', 'City', 'Customer type', 'Gender', 'Product line', 'Payment']

for var in categorical_vars:
    print(f"\n{var}:")
    value_counts = df[var].value_counts()
    for value, count in value_counts.items():
        percentage = (count / len(df)) * 100
        print(f"  {value}: {count:,} ({percentage:.1f}%)")


## 3. Distribution Analysis


In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

key_variables = ['Total', 'Unit price', 'Quantity', 'Rating', 'Tax 5%', 'gross income']

for i, var in enumerate(key_variables):
    axes[i].hist(df[var].dropna(), bins=30, alpha=0.7, edgecolor='black')
    axes[i].set_title(f'Distribution of {var}')
    axes[i].set_xlabel(var)
    axes[i].set_ylabel('Frequency')
    
    mean_val = df[var].mean()
    median_val = df[var].median()
    axes[i].axvline(mean_val, color='red', linestyle='--', label=f'Mean: {mean_val:.2f}')
    axes[i].axvline(median_val, color='green', linestyle='--', label=f'Median: {median_val:.2f}')
    axes[i].legend()

plt.tight_layout()
plt.show()


In [ ]:
print("=== NORMALITY TESTS ===")
print("\nShapiro-Wilk Test for Normality (sample size < 5000):")
print("H0: Data is normally distributed")
print("H1: Data is not normally distributed")
print("\nResults:")

for var in key_variables:
    sample_data = df[var].dropna().sample(min(5000, len(df[var].dropna())))
    
    if len(sample_data) > 3:  # Minimum sample size for Shapiro-Wilk
        stat, p_value = stats.shapiro(sample_data)
        print(f"{var}: Statistic={stat:.4f}, p-value={p_value:.4f}")
        if p_value < 0.05:
            print(f"  → Reject H0: {var} is NOT normally distributed")
        else:
            print(f"  → Fail to reject H0: {var} appears normally distributed")
    print()


## 4. Correlation Analysis


In [ ]:
correlation_matrix = df[numeric_columns].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.3f', cbar_kws={'shrink': 0.8})
plt.title('Correlation Matrix of Numeric Variables')
plt.tight_layout()
plt.show()


## 5. Hypothesis Testing


In [ ]:
print("=== HYPOTHESIS TESTING ===")
print("\n1. Gender Differences in Total Spending:")
print("H0: No difference in mean spending between genders")
print("H1: There is a difference in mean spending between genders")

male_spending = df[df['Gender'] == 'Male']['Total']
female_spending = df[df['Gender'] == 'Female']['Total']

print(f"\nMale spending: Mean = ${male_spending.mean():.2f}, Std = ${male_spending.std():.2f}")
print(f"Female spending: Mean = ${female_spending.mean():.2f}, Std = ${female_spending.std():.2f}")

t_stat, p_value = ttest_ind(male_spending, female_spending)
print(f"\nT-test results:")
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("→ Reject H0: There IS a significant difference in spending between genders")
else:
    print("→ Fail to reject H0: No significant difference in spending between genders")

pooled_std = np.sqrt(((len(male_spending) - 1) * male_spending.var() + 
                     (len(female_spending) - 1) * female_spending.var()) / 
                    (len(male_spending) + len(female_spending) - 2))
cohens_d = (male_spending.mean() - female_spending.mean()) / pooled_std
print(f"\nEffect size (Cohen's d): {cohens_d:.4f}")
if abs(cohens_d) < 0.2:
    print("→ Small effect size")
elif abs(cohens_d) < 0.5:
    print("→ Medium effect size")
else:
    print("→ Large effect size")


In [ ]:
print("\n\n2. Chi-Square Tests for Categorical Associations:")

print("\nGender vs Product Line:")
print("H0: Gender and Product line are independent")
print("H1: Gender and Product line are associated")

gender_product_crosstab = pd.crosstab(df['Gender'], df['Product line'])
chi2, p_value, dof, expected = chi2_contingency(gender_product_crosstab)

print(f"Chi-square statistic: {chi2:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Degrees of freedom: {dof}")

if p_value < 0.05:
    print("→ Reject H0: Gender and Product line ARE associated")
else:
    print("→ Fail to reject H0: Gender and Product line are independent")


print("\n\nCustomer Type vs Payment Method:")
print("H0: Customer type and Payment method are independent")
print("H1: Customer type and Payment method are associated")

customer_payment_crosstab = pd.crosstab(df['Customer type'], df['Payment'])
chi2, p_value, dof, expected = chi2_contingency(customer_payment_crosstab)

print(f"Chi-square statistic: {chi2:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Degrees of freedom: {dof}")

if p_value < 0.05:
    print("→ Reject H0: Customer type and Payment method ARE associated")
else:
    print("→ Fail to reject H0: Customer type and Payment method are independent")


## 6. Summary and Conclusions


In [ ]:
print("=== STATISTICAL ANALYSIS SUMMARY ===")
print(f"\nDataset Overview:")
print(f"• Total transactions: {len(df):,}")
print(f"• Total revenue: ${df['Total'].sum():,.2f}")
print(f"• Average transaction value: ${df['Total'].mean():.2f}")
print(f"• Date range: {df['Date'].min().strftime('%Y-%m-%d')} to {df['Date'].max().strftime('%Y-%m-%d')}")

print(f"\nKey Findings:")
print(f"• Peak revenue day: {df.groupby('day_name')['Total'].sum().idxmax()}")
print(f"• Peak revenue hour: {df.groupby('hour')['Total'].sum().idxmax()}:00")
print(f"• Most popular product line: {df['Product line'].value_counts().index[0]}")
print(f"• Most common payment method: {df['Payment'].value_counts().index[0]}")
print(f"• Gender distribution: {df['Gender'].value_counts().to_dict()}")
print(f"• Customer type distribution: {df['Customer type'].value_counts().to_dict()}")

print(f"\nBusiness Insights:")
print(f"• The dataset shows significant patterns in customer behavior")
print(f"• Time-based factors (hour, day) are important predictors of spending")
print(f"• Product line and customer demographics influence transaction values")
print(f"• Statistical tests reveal meaningful differences across customer segments")
